In [36]:
# Import os
import requests
import openai
from dotenv import load_dotenv


In [37]:
# Map moods to TMDB genre IDs
mood_to_genre = {
    'happy': ['35', '10751'],        # Comedy, Family
    'sad': ['18'],                  # Drama
    'excited': ['28', '12'],        # Action, Adventure
    'romantic': ['10749'],          # Romance
    'scared': ['27', '53'],         # Horror, Thriller
    'curious': ['99'],              # Documentary
    'fantasy': ['14', '16'],        # Fantasy, Animation
    'mysterious': ['9648'],         # Mystery
    'inspired': ['36', '10402'],    # History, Music
    'sci-fi': ['878'],              # Science Fiction
}

def get_genres_for_mood(mood):
    """Return a list of genre IDs for a given mood."""
    return mood_to_genre.get(mood.lower(), ['18'])  # Default to Drama if not found

In [38]:
def fetch_movies_from_tmdb(mood, decade, min_rating, country):
    """Fetch movies from TMDB based on mood, decade, minimum rating, and country of origin."""
    genres = get_genres_for_mood(mood)
    genre_str = ','.join(genres)
    start_date = f'{decade}-01-01'
    end_date = f'{decade+9}-12-31'
    load_dotenv()
    api_key = os.getenv('TMDB_API_KEY')
    url = "https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "with_genres": genre_str,
        "sort_by": "popularity.desc",
        "primary_release_date.gte": start_date,
        "primary_release_date.lte": end_date,
        "vote_average.gte": min_rating,
        "region": country.upper(),
    }
    response = requests.get(url, params=params)
    return response.json().get("results", [])

In [ ]:
# Load OpenAI API key from .env file (add OPENAI_API_KEY=your_key to .env)
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

def generate_movie_recommendations(mood, movies, n=3):
    """Use OpenAI GPT to recommend and describe n movies for the given mood, showing TMDB rating next to each film name."""
    if not openai_api_key:
        raise ValueError('OpenAI API key not found. Please add OPENAI_API_KEY to your .env file.')
    openai.api_key = openai_api_key
    movie_list = '\n'.join([f"- {m['title']} (TMDB rating: {m.get('vote_average', 'N/A')}): {m.get('overview', 'No description available.')}" for m in movies[:10]])
    prompt = (f"User mood: {mood}\n"
              f"Here are some movies fetched from TMDB that might fit this mood (with their TMDB ratings):\n{movie_list}\n\n"
              f"Please recommend and describe {n} movies from this list that best fit the user's mood. For each recommended movie, you must display the TMDB rating next to the movie title in your output. Respond in a friendly, engaging way.")
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.7,
    )
    return response.choices[0].message.content

In [40]:
# Example usage: fetch movies and generate recommendations using the function
mood = 'sad'  # e.g., 'sad', 'excited', etc.
decade = 2000  # e.g., 1980, 1990, 2000, 2010, 2020
min_rating = 8.0  # Minimum average vote
country = 'US'    # 2-letter country code

movies = fetch_movies_from_tmdb(mood, decade, min_rating, country)

In [41]:
# Generate and display AI-powered recommendations
output = generate_movie_recommendations(mood, movies, n=10)
print(output)

I'm sorry to hear that you're feeling sad. Here are 10 movies from the list that might resonate with your mood:

1. The Dark Knight: This intense and gripping film delves into the darkness of Gotham City, where chaos reigns and the line between good and evil becomes blurred.

2. Inglourious Basterds: A tale of revenge and resistance during WWII, this film showcases the brutality of war and the lengths people will go to in order to fight back against oppression.

3. Gladiator: A story of betrayal and redemption, Gladiator follows the journey of a fallen warrior who seeks vengeance against those who wronged him.

4. Oldboy: This haunting and twisted tale of revenge will keep you on the edge of your seat as the protagonist seeks justice for the years of torment he endured.

5. The Departed: A gritty and suspenseful crime drama, The Departed explores the complexities of loyalty and betrayal within the criminal underworld.

6. Eternal Sunshine of the Spotless Mind: A poignant and thought-pr